# Google Calendar API

https://developers.google.com/calendar/quickstart/python

In [1]:
"""
Shows basic usage of the Google Calendar API. Creates a Google Calendar API
service object and outputs a list of the next 10 events on the user's calendar.
"""
from __future__ import print_function
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import datetime

In [2]:
# Setup the Calendar API
SCOPES = 'https://www.googleapis.com/auth/calendar.readonly'
store = file.Storage('credentials.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('calendar', 'v3', http=creds.authorize(Http()))

In [3]:
# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                      maxResults=10, singleEvents=True,
                                      orderBy='startTime').execute()
events = events_result.get('items', [])

Getting the upcoming 10 events


In [4]:
if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])

2018-07-06T10:15:00+02:00 Sandu@work


In [5]:
import pytz
import re

In [6]:
page_token = None
while True:
    calendar_list = service.calendarList().list(pageToken=page_token).execute()
    for calendar_list_entry in calendar_list['items']:
        print(calendar_list_entry['summary'])
        
    page_token = calendar_list.get('nextPageToken')
    if not page_token:
        break

edgar@bellmetric.net
asser@bellmetric.net
jakub@bellmetric.net
sandu@bellmetric.net
christoffer@bellmetric.net
oana@bellmetric.net
janus@bellmetric.net
mads-emil@bellmetric.net
maximilian@bellmetric.net
marcus@bellmetric.net
Contacts
Holidays in Denmark
Week Numbers


In [7]:
items = []
for calendar_list_entry in calendar_list['items']:
    if '@' in calendar_list_entry['summary']:
        items.append({'id': calendar_list_entry['summary']})

In [8]:
timezone = 'Europe/Copenhagen'
body = {
    "timeMin": now,
    "timeMax": pytz.timezone(timezone).localize(datetime.datetime.strptime((datetime.datetime.now() + 
            datetime.timedelta(1)).isoformat().split('T')[0], '%Y-%m-%d')).isoformat(),
    "timeZone": timezone,
#     "groupExpansionMax": integer,
#     "calendarExpansionMax": integer,
    "items": items
}

In [9]:
freebusy = service.freebusy().query(body=body).execute()

In [10]:
for empl in freebusy['calendars']:
    if freebusy['calendars'][empl]['busy'] != []:
        # Deal with vacations
        event = service.events().list(calendarId=empl, timeMin=now,
                                      maxResults=1, singleEvents=True,
                                      orderBy='startTime').execute()
        try:
            td = (datetime.datetime.strptime(event['items'][0]['end']['date'],'%Y-%m-%d') - \
                  datetime.datetime.strptime(event['items'][0]['start']['date'],'%Y-%m-%d'))
        except:
            td = (datetime.datetime.strptime(event['items'][0]['end']['dateTime'].split('T')[0],'%Y-%m-%d') - \
                  datetime.datetime.strptime(event['items'][0]['start']['dateTime'].split('T')[0],'%Y-%m-%d'))
        
        if td.days == 0:
            print(empl.split('@')[0].capitalize(),'\t-->\t', re.search('(?<=T).....', 
                freebusy['calendars'][empl]['busy'][0]['end']).group())

Sandu 	-->	 18:15
